In [26]:
import seaborn as sns

from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.preprocessing import sequence
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras import regularizers
from keras import optimizers
from keras.callbacks import EarlyStopping
from keras.layers import Embedding, Conv1D, MaxPooling1D, GlobalMaxPooling1D 
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from keras.layers import Embedding

from tqdm import tqdm
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer 
import os, re, csv, math, codecs


import pandas as pd
import numpy as np
import re

import matplotlib.pyplot as plt

In [27]:
feature_df = pd.read_csv('./cleaned_csv/feature_df.csv',low_memory=False)
print('rows:',len(feature_df))

rows: 239850


In [28]:
feature_df

,sentiment,cleaned_text,original_word_count,original_average_word_length,stopword_ratio
0,Neutral,big group make reservation week advance,0.120370,0.055914,0.533333
1,Neutral,wait 20 minute drink another 30 breakfast not ...,0.111111,0.081413,0.285714
2,Neutral,veggie cheese steak russian coleslaw like vega...,0.092593,0.086022,0.333333
3,Neutral,essential climb rope harness climb shoe beer,0.064815,0.107527,0.111111
4,Neutral,cleanest asian grocery store spotless organize...,0.175926,0.098310,0.190476
...,...,...,...,...,...
239845,Poor,lazy as bitch,0.037037,0.060932,0.333333
239846,Poor,fastest wing ever hit table4 minute flat,0.064815,0.090800,0.222222
239847,Poor,worst service warm beer girl never clue 18 u o...,0.287037,0.059954,0.363636
239848,Poor,delivery faster expect,0.037037,0.086022,0.500000


In [29]:
feature_df.dropna(inplace = True)

In [30]:
print(feature_df.shape)

feature_df.head()

(239598, 5)


,sentiment,cleaned_text,original_word_count,original_average_word_length,stopword_ratio
0,Neutral,big group make reservation week advance,0.120370,0.055914,0.533333
1,Neutral,wait 20 minute drink another 30 breakfast not ...,0.111111,0.081413,0.285714
2,Neutral,veggie cheese steak russian coleslaw like vega...,0.092593,0.086022,0.333333
3,Neutral,essential climb rope harness climb shoe beer,0.064815,0.107527,0.111111
4,Neutral,cleanest asian grocery store spotless organize...,0.175926,0.098310,0.190476


In [31]:
#feature_df = feature_df.sample(n=10000)

In [32]:
#create dummy variables

In [33]:
feature_df = feature_df[['sentiment','cleaned_text']]

In [34]:
dummies = pd.get_dummies(feature_df['sentiment'])

In [35]:
atom_col = [c for c in dummies.columns if '*' not in c]


In [36]:
for col in atom_col:
    feature_df[col] = dummies[[c for c in dummies.columns if col in c]].sum(axis=1)

<ipython-input-36-e513d7692cb6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feature_df[col] = dummies[[c for c in dummies.columns if col in c]].sum(axis=1)


In [37]:
feature_df = feature_df.drop(columns = ["sentiment"])

In [38]:
feature_df

,cleaned_text,Good,Neutral,Poor
0,big group make reservation week advance,0,1,0
1,wait 20 minute drink another 30 breakfast not ...,0,1,0
2,veggie cheese steak russian coleslaw like vega...,0,1,0
3,essential climb rope harness climb shoe beer,0,1,0
4,cleanest asian grocery store spotless organize...,0,1,0
...,...,...,...,...
239845,lazy as bitch,0,0,1
239846,fastest wing ever hit table4 minute flat,0,0,1
239847,worst service warm beer girl never clue 18 u o...,0,0,1
239848,delivery faster expect,0,0,1


In [39]:
feature_df_comments_labels = feature_df[["Poor", "Neutral", "Good"]]
feature_df_comments_labels.head()

,Poor,Neutral,Good
0,0,1,0
1,0,1,0
2,0,1,0
3,0,1,0
4,0,1,0


# Multi-lable Text Classification Model with Single Output Layer

In [40]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [41]:
X = []
sentences = list(feature_df["cleaned_text"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = feature_df_comments_labels.values

In [42]:
#train test split

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

In [44]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [45]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

fast_file = open('wiki-news-300d-1M.vec', encoding="utf8")

for line in fast_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
fast_file.close()

embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

#### LSTM

In [21]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(3, activation='softmax')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [22]:
print(model.summary())

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 300)          13753200  
_________________________________________________________________
lstm (LSTM)                  (None, 128)               219648    
_________________________________________________________________
dense (Dense)                (None, 3)                 387       
Total params: 13,973,235
Trainable params: 220,035
Non-trainable params: 13,753,200
_________________________________________________________________
None


In [23]:
#conda install -c anaconda pydot

In [24]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot4a.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [25]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=1, validation_split=0.2)


Epoch 1/5
1198/1198 [==============================] - 684s 571ms/step - loss: 0.6366 - acc: 0.3339 - val_loss: 0.6365 - val_acc: 0.3323
Epoch 2/5
1198/1198 [==============================] - 755s 630ms/step - loss: 0.6365 - acc: 0.3336 - val_loss: 0.6365 - val_acc: 0.3323
Epoch 3/5
 287/1198 [======>.......................] - ETA: 7:43 - loss: 0.6365 - acc: 0.3339

KeyboardInterrupt: 

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])

plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train','test'], loc='upper left')
plt.show()

# Multi-lable Text Classification Model with Multiple Output Layers

In [46]:
def preprocess_text(sen):
    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)

    return sentence

In [47]:
X = []
sentences = list(feature_df["cleaned_text"])
for sen in sentences:
    X.append(preprocess_text(sen))

y = feature_df[["Poor","Neutral","Good"]]

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=123)

In [49]:
# First output
y1_train = y_train[["Poor"]].values
y1_test =  y_test[["Poor"]].values

# Second output
y2_train = y_train[["Neutral"]].values
y2_test =  y_test[["Neutral"]].values

# Third output
y3_train = y_train[["Good"]].values
y3_test =  y_test[["Good"]].values

In [50]:
#convert textual inputs to embedded vectors

In [51]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [52]:
#word embedding

In [53]:
fast_file = open('wiki-news-300d-1M.vec', encoding="utf8")

for line in fast_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
fast_file.close()

embedding_matrix = zeros((vocab_size, 300))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [54]:
#LSTM

In [55]:
input_1 = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 300, weights=[embedding_matrix], trainable=False)(input_1)
LSTM_Layer1 = LSTM(128)(embedding_layer)

output1 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output2 = Dense(1, activation='sigmoid')(LSTM_Layer1)
output3 = Dense(1, activation='sigmoid')(LSTM_Layer1)


model = Model(inputs=input_1, outputs=[output1, output2, output3])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])

In [56]:
print(model.summary())

Model: "functional_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 200)]        0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 300)     13753200    input_2[0][0]                    
__________________________________________________________________________________________________
lstm_1 (LSTM)                   (None, 128)          219648      embedding_1[0][0]                
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1)            129         lstm_1[0][0]                     
_______________________________________________________________________________________

In [57]:
from keras.utils import plot_model
plot_model(model, to_file='model_plot4b.png', show_shapes=True, show_layer_names=True)

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [58]:
history = model.fit(x=X_train, y=[y1_train, y2_train, y3_train], batch_size=8192, epochs=5, verbose=1, validation_split=0.2)

Epoch 1/5
19/19 [==============================] - 811s 43s/step - loss: 2.0177 - dense_1_loss: 0.6705 - dense_2_loss: 0.6745 - dense_3_loss: 0.6728 - dense_1_acc: 0.6648 - dense_2_acc: 0.6684 - dense_3_acc: 0.6669 - val_loss: 1.9872 - val_dense_1_loss: 0.6654 - val_dense_2_loss: 0.6593 - val_dense_3_loss: 0.6625 - val_dense_1_acc: 0.6678 - val_dense_2_acc: 0.6645 - val_dense_3_acc: 0.6677
Epoch 2/5
19/19 [==============================] - 801s 42s/step - loss: 1.9563 - dense_1_loss: 0.6548 - dense_2_loss: 0.6485 - dense_3_loss: 0.6530 - dense_1_acc: 0.6648 - dense_2_acc: 0.6684 - dense_3_acc: 0.6669 - val_loss: 1.9117 - val_dense_1_loss: 0.6369 - val_dense_2_loss: 0.6380 - val_dense_3_loss: 0.6368 - val_dense_1_acc: 0.6678 - val_dense_2_acc: 0.6645 - val_dense_3_acc: 0.6677
Epoch 3/5
19/19 [==============================] - 875s 46s/step - loss: 1.9122 - dense_1_loss: 0.6384 - dense_2_loss: 0.6366 - dense_3_loss: 0.6371 - dense_1_acc: 0.6648 - dense_2_acc: 0.6684 - dense_3_acc: 0.6669

In [59]:
score = model.evaluate(x=X_test, y=[y1_test, y2_test, y3_test], verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])

1498/1498 [==============================] - 184s 123ms/step - loss: 1.9096 - dense_1_loss: 0.6341 - dense_2_loss: 0.6369 - dense_3_loss: 0.6387 - dense_1_acc: 0.6703 - dense_2_acc: 0.6662 - dense_3_acc: 0.6635
Test Score: 1.9096430540084839
Test Accuracy: 0.6341055035591125
